In [ ]:
import sys

sys.path.insert(0, "../")

In [ ]:
import rasterio
from rasterio.plot import show
from cerulean_cloud.cloud_run_orchestrator.clients import *
from cerulean_cloud.cloud_run_orchestrator.handler import (
    offset_group_shape_from_base_tiles,
    group_bounds_from_list_of_bounds,
)
from cerulean_cloud.titiler_client import TitilerClient
from cerulean_cloud.tiling import TMS as tiler
from cerulean_cloud.tiling import offset_bounds_from_base_tiles

In [ ]:
import os

os.environ["API_KEY"] = "your-api-key-value"

In [ ]:
TITILER_URL = "https://iwvu2v4r5i.execute-api.eu-central-1.amazonaws.com/"
INFERENCE_URL = (
    "https://cerulean-cloud-staging-cr-offset-tiles-49b-5qkjkyomta-ew.a.run.app"
)
AUX_INFRA_DISTANCE = "gs://ceruleanml/aux_datasets/infra_locations_01_cogeo.tiff"
scene_id = "S1A_IW_GRDH_1SDV_20230726T183302_20230726T183327_049598_05F6CA_31E7"  # XXX crosses antimeridian!
#

In [ ]:
titiler_client = TitilerClient(url=TITILER_URL)
bounds = await titiler_client.get_bounds(scene_id)
stats = await titiler_client.get_statistics(scene_id, band="vv")
base_tiles = list(tiler.tiles(*bounds, [9], truncate=False))
offset_image_shape = offset_group_shape_from_base_tiles(base_tiles, scale=2)
offset_tiles_bounds = offset_bounds_from_base_tiles(base_tiles)
offset_bounds = group_bounds_from_list_of_bounds(offset_tiles_bounds)

In [ ]:
bounds

In [ ]:
client = CloudRunInferenceClient(
    url=INFERENCE_URL,
    titiler_client=titiler_client,
    scene_id=scene_id,
    tileset_envelope_bounds=offset_bounds,
    offset_image_shape=offset_image_shape,
    aux_datasets=["ship_density", AUX_INFRA_DISTANCE],
)

In [ ]:
client.aux_datasets

In [ ]:
with client.aux_datasets.open() as ds:
    with rasterio.open("aux_ds.tiff", **ds.profile, mode="w") as dst:
        dst.write(ds.read())

In [ ]:
bounds

In [ ]:
offset_image_shape

In [ ]:
offset_bounds

In [ ]:
import supermercado

tiles_np = np.array([(tile.x, tile.y, tile.z) for tile in base_tiles])
tilexmin, tilexmax, tileymin, tileymax = supermercado.super_utils.get_range(tiles_np)

In [ ]:
tilexmin, tilexmax, tileymin, tileymax